In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import time
import warnings
warnings.filterwarnings('ignore')

enc = OneHotEncoder()
path = 'data/'
train = pd.read_csv(path+'train.csv')
vali = pd.read_csv(path+'validation.csv')
test = pd.read_csv(path+'test.csv')
start_time = time.time()

In [3]:
basectr = 0.0007539649884458758#train['click'].value_counts(normalize = True)[1]

In [4]:
ad_ctr = [0.00083473226564105031,
 0.00050464562342478474,
 0.00074884754565216914,
 0.00076448084204447765,
 0.00030658545558598702,
 0.00062225602378400803,
 0.0007180766777520589,
 0.0046066034099877034,
 0.00030675100937663219]#[]
ad_name = [1458,3476,3427,3358,2259,2821,3386,2997,2261]
# for i in ad_name:
#     ad_ctr.append(train['click'][train['advertiser']==i].value_counts(normalize = True)[1])
ad_len = len(ad_name)

In [6]:
train_onehot = pd.read_csv('train_onehot.csv')
vali_onehot = pd.read_csv('vali_onehot.csv')

In [7]:
import pickle
rf_model = pickle.load(open('rf_model.sav','rb'))

In [8]:
names =[str(x) for x in list(vali_onehot.columns)]
importance = pd.DataFrame({'Importance0':rf_model[0].feature_importances_,'Importance1':rf_model[1].feature_importances_,'Importance2':rf_model[2].feature_importances_,
             'Importance3':rf_model[3].feature_importances_,'Importance4':rf_model[4].feature_importances_,
              'Importance5':rf_model[5].feature_importances_,'Importance6':rf_model[6].feature_importances_,
              'Importance7':rf_model[7].feature_importances_,'Importance8':rf_model[8].feature_importances_})

In [9]:
importance['importance_sum'] = importance.apply(lambda x: x.sum(),axis=1)
importance['Names'] = names

In [10]:
vali_ad_onehot = []
for i in ad_name:
    vali_ad_onehot.append(vali_onehot[vali_onehot["advertiser_"+str(i)]==1]) 

In [11]:
zero_importance_rf = importance[importance['importance_sum']!=0].Names.values
remove_list=names

for x in zero_importance_rf:
    remove_list.remove(x)

In [12]:
def dataprocessing (data,data_onehot):
    slotprice = data['slotprice']
    bidprice = data['bidprice']
    payprice = data['payprice']
    click = data['click']
    
    ad_onehot = []
    
    ad_onehot = []
    for i in ad_name:
        ad_onehot.append(data_onehot[data_onehot["advertiser_"+str(i)]==1]) 
    
    ad_click = []
    for i in ad_name:
        ad_click.append(data['click'][data['advertiser']==i])
        
    return ad_onehot, ad_click, slotprice, bidprice, payprice

In [13]:
train_ad_onehot,train_ad_click,train_slotprice,train_bidprice,train_payprice = dataprocessing(train,train_onehot)
vali_ad_onehot,vali_ad_click,vali_slotprice,vali_bidprice,vali_payprice = dataprocessing(vali,vali_onehot)

In [15]:
train_click = train[['click','advertiser']]
vali_ad_onehot_xgb= train_onehot.drop(remove_list,axis=1)
train_ad_onehot_xgb= vali_onehot.drop(remove_list,axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_onehot, train_click, test_size=0.2, random_state=42)

In [16]:
def dataprocessing_ (data_onehot,click):
    ad_onehot = []
    for i in ad_name:
        ad_onehot.append(data_onehot[data_onehot["advertiser_"+str(i)]==1]) 
#     print('done')
    ad_click = []
    for i in ad_name:
#         print(type(click))
        a = click[click['advertiser']==i]
        a = a.drop('advertiser',1)
#         print(a.head())
        ad_click.append(a)    
    return ad_onehot, ad_click

In [17]:
_train_ad_onehot,_train_ad_click = dataprocessing_(X_train,y_train)
_ad_onehot,_ad_click= dataprocessing_(X_test,y_test)

In [18]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import classification_report as report

In [19]:
import xgboost as xgb

xgb_model = []
train_ad_pd = []
train_ad_rmse = []
train_ad_auc = []
train_ad_report = []
vali_ad_pd = []
vali_ad_rmse = []
vali_ad_auc = []

param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
plst = param.items()

num_round = 10

for i in range(ad_len):
    dtrain = (xgb.DMatrix(_train_ad_onehot[i],_train_ad_click[i]))
    dvali = (xgb.DMatrix(_ad_onehot[i],_ad_click[i]))
    evallist  = [(dvali,'eval'), (dtrain,'train')]
    dptrain = xgb.DMatrix(train_ad_onehot[i])
    dpvali =xgb.DMatrix(vali_ad_onehot[i])
    
    xgb_model.append(xgb.train( plst, dtrain, num_round, evallist ))
    train_ad_pd.append(xgb_model[i].predict(dptrain))
    train_ad_rmse.append(rmse(train_ad_click[i],train_ad_pd[i]))
    train_ad_auc.append(auc(train_ad_click[i],train_ad_pd[i]))
    
    vali_ad_pd.append(xgb_model[i].predict(dpvali))
    vali_ad_rmse.append(rmse(vali_ad_click[i],vali_ad_pd[i]))
    vali_ad_auc.append(auc(vali_ad_click[i],vali_ad_pd[i]))
    print(i+1,'/9')

[0]	eval-auc:0.874726	train-auc:0.838692
[1]	eval-auc:0.874764	train-auc:0.838739
[2]	eval-auc:0.893065	train-auc:0.872415
[3]	eval-auc:0.924495	train-auc:0.91202
[4]	eval-auc:0.950284	train-auc:0.943447
[5]	eval-auc:0.952522	train-auc:0.946195
[6]	eval-auc:0.959695	train-auc:0.950249
[7]	eval-auc:0.958682	train-auc:0.959249
[8]	eval-auc:0.957698	train-auc:0.964258
[9]	eval-auc:0.955095	train-auc:0.966611
1 /9
[0]	eval-auc:0.5	train-auc:0.5
[1]	eval-auc:0.723147	train-auc:0.829556
[2]	eval-auc:0.723071	train-auc:0.831126
[3]	eval-auc:0.731619	train-auc:0.839873
[4]	eval-auc:0.728242	train-auc:0.866599
[5]	eval-auc:0.727653	train-auc:0.872377
[6]	eval-auc:0.772911	train-auc:0.914038
[7]	eval-auc:0.815196	train-auc:0.935729
[8]	eval-auc:0.797808	train-auc:0.949348
[9]	eval-auc:0.811628	train-auc:0.952915
2 /9
[0]	eval-auc:0.776993	train-auc:0.821568
[1]	eval-auc:0.777002	train-auc:0.82173
[2]	eval-auc:0.795036	train-auc:0.85223
[3]	eval-auc:0.797125	train-auc:0.853402
[4]	eval-auc:0.8797

In [20]:
j = 0
print('xgb validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_rmse[j],'AUC',vali_ad_auc[j])
    j+=1

xgb validation results
ad 1458 : RMSE 0.0217942473851 AUC 0.964258274281
ad 3476 : RMSE 0.019836956863 AUC 0.823949256842
ad 3427 : RMSE 0.0247393551685 AUC 0.952029031574
ad 3358 : RMSE 0.0227416115309 AUC 0.920750748383
ad 2259 : RMSE 0.0259010117117 AUC 0.597219759762
ad 2821 : RMSE 0.0250923827508 AUC 0.530942574953
ad 3386 : RMSE 0.0245510455235 AUC 0.668235845393
ad 2997 : RMSE 0.0655661704949 AUC 0.661435393834
ad 2261 : RMSE 0.0193977560541 AUC 0.494605312383


In [21]:
best = [0.964258274281,0.823949256842,0.952029031574,0.937509717487,0.656032534019,0.665704540131,0.729408271574,0.661435393834,0.773310886644]
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_auc)))

[4.9515946898281982e-14,
 -1.6364687382974807e-13,
 -4.8605564018089353e-13,
 0.016758969103661037,
 0.058812774256578049,
 0.13476196517784578,
 0.06117242618136487,
 1.3744561044859438e-13,
 0.27870557426090992]

In [22]:
train_ad_onehot[0].shape

(540293, 280)

In [23]:
for i in range(len(train_ad_onehot)):
    vali_ad_onehot[i] = vali_ad_onehot[i].drop(remove_list,axis=1)
    train_ad_onehot[i] = train_ad_onehot[i].drop(remove_list,axis=1)

In [24]:
train_ad_onehot[0].shape

(540293, 259)

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
gbc_model = []
train_ad_gbcpd = []
train_ad_gbcrmse = []
train_ad_gbcauc = []
vali_ad_gbcpd = []
vali_ad_gbcrmse = []
vali_ad_gbcauc = []
    
for i in range(ad_len):

    gbc_model.append(GradientBoostingClassifier(n_estimators= 100,max_depth= 4, min_samples_leaf=15, learning_rate= 0.035))
#     print('fitting model:',i)
    gbc_model[i].fit(train_ad_onehot[i], train_ad_click[i])
#     print('predicting')
    vali_ad_gbcpd.append((gbc_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_gbcrmse.append(rmse(vali_ad_click[i],vali_ad_gbcpd[i]))
    vali_ad_gbcauc.append(auc(vali_ad_click[i],vali_ad_gbcpd[i]))
    print(i+1,'/9')

1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [29]:
j = 0
print('extra tree classification validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_gbcrmse[j],'AUC',vali_ad_gbcauc[j])
    j+=1

extra tree classification validation results
ad 1458 : RMSE 0.0298038599796 AUC 0.963225343893
ad 3476 : RMSE 0.0207179601668 AUC 0.672073432157
ad 3427 : RMSE 0.0285043446949 AUC 0.694419500954
ad 3358 : RMSE 0.0414943462695 AUC 0.937509717487
ad 2259 : RMSE 0.0259830818523 AUC 0.591042063738
ad 2821 : RMSE 0.0250796616159 AUC 0.58919962133
ad 3386 : RMSE 0.024527083622 AUC 0.68457227863
ad 2997 : RMSE 0.0655184049188 AUC 0.656547680016
ad 2261 : RMSE 0.0211107043447 AUC 0.57011597456


In [30]:
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_gbcauc)))#0.00005,0.00003#0.00004

[0.0010329303877112395,
 0.15187582468511862,
 0.25760953062040814,
 3.4172664697962318e-13,
 0.064990470280690493,
 0.076504918800581523,
 0.044835992944110337,
 0.0048877138176114387,
 0.20319491208358098]

In [31]:
from sklearn.ensemble import AdaBoostClassifier
adb_model = []
train_ad_adbpd = []
train_ad_adbrmse = []
train_ad_adbauc = []
vali_ad_adbpd = []
vali_ad_adbrmse = []
vali_ad_adbauc = []

for i in range(ad_len):
    adb_model.append(AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=2017))
    adb_model[i].fit(train_ad_onehot[i], train_ad_click[i])
#     train_ad_adbpd.append((adb_model[i].predict_proba(train_ad_onehot[i]))[:,1])
#     train_ad_adbrmse.append(rmse(train_ad_click[i],train_ad_adbpd[i]))
#     train_ad_adbauc.append(auc(train_ad_adbpd[i],train_ad_adbpd[i]))
    vali_ad_adbpd.append((adb_model[i].predict_proba(vali_ad_onehot[i]))[:,1])
    vali_ad_adbrmse.append(rmse(vali_ad_click[i],vali_ad_adbpd[i]))
    vali_ad_adbauc.append(auc(vali_ad_click[i],vali_ad_adbpd[i]))
    print(i+1,'/9')


1 /9
2 /9
3 /9
4 /9
5 /9
6 /9
7 /9
8 /9
9 /9


In [32]:
j = 0
print('Adaboosting validation results')
for i in ad_name:    
    print('ad',i,':','RMSE',vali_ad_adbrmse[j],'AUC',vali_ad_adbauc[j])
    j+=1

Adaboosting validation results
ad 1458 : RMSE 0.450369772464 AUC 0.963908628595
ad 3476 : RMSE 0.449349903403 AUC 0.674023949059
ad 3427 : RMSE 0.453485636704 AUC 0.942543220668
ad 3358 : RMSE 0.449437385353 AUC 0.93714401151
ad 2259 : RMSE 0.416637456771 AUC 0.592787332122
ad 2821 : RMSE 0.452913962076 AUC 0.591012453154
ad 3386 : RMSE 0.459870000249 AUC 0.665648746772
ad 2997 : RMSE 0.466973307183 AUC 0.630300624808
ad 2261 : RMSE 0.403631403828 AUC 0.569726898616


In [33]:
list(map(lambda x: x[0]-x[1], zip(best, vali_ad_adbauc)))#0.00005,0.00003#0.00004

[0.00034964568575202026,
 0.14992530778328039,
 0.0094858109057793794,
 0.00036570597711604869,
 0.063245201897196845,
 0.074692086976721361,
 0.063759524802036616,
 0.031134769026051612,
 0.20358398802821254]